## Project 1
### Sneha Ganganna (5579362)

In [1]:
#To check if the spark has been installed 
!pip install pyspark
import pyspark



import csv
import time
from pyspark import SparkConf
from pyspark.sql.types import *
from collections import Counter
from pyspark import SparkContext
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
from pyspark.sql.functions import countDistinct
#from pyspark.sql.functions import col


from pyspark.sql.functions import col, udf,explode, split, desc
from pyspark.sql.types import StringType
from collections import Counter



In [2]:

# Spark Session initialisation
sparkSession = SparkSession.builder.appName("Project 1")\
    .config("spark.memory.fraction", 0.8) \
    .config("spark.driver.cores", 8) \
    .config("spark.executor.memory", "16g") \
    .config("spark.driver.memory", "16g")\
    .config("spark.sql.shuffle.partitions" , "1000") \
    .getOrCreate()
sc = sparkSession.sparkContext

#Exercise 1. 2 (Loading the dataset into an RDD)

In [3]:


# Read users_libraries file into RDD
userRatingsRDD = sc.textFile("users_libraries")
# Create Pair RDD for users_libraries Text File
userRatingsRDD = sc.textFile("users_libraries.txt").map(lambda line: line.split(";")).map(lambda line: (line[0],list(map(int,line[1].split(",")))))

#PaperCsv RDD
def processCsvFile(line):
    paper = csv.reader([line.replace("\0", "")], delimiter=',', quoting=csv.QUOTE_MINIMAL)
    paperList = next(paper)
    return paperList[0], paperList[14]

paperTermsRDD = sc.textFile("papers.csv")
paperTermsRDD = paperTermsRDD.map(processCsvFile).filter(lambda x: (x[1] != "" and x[1] != " ")).map(lambda x: (int(x[0]),x[1].split(" ")))

# Join the two RDDs on the common key i.e. paper_id
#userLibrariesJoinPapersRdd = userRatingsRDD.flatMapValues(lambda x: x).map(lambda x: (x[1],x[0])).join(paperTermsRDD)

# Print the result
#paperTerms_userRatings.foreach(print)
#paperTerms_userRatings.foreach(lambda x: print(x))

In [4]:

#to record execution time
start_time = time.time()

#load stopwords_en file
stopWords = sc.textFile("stopwords_en.txt")
stopWordsBroadcast = sc.broadcast(stopWords.collect())

#print("Stop words Start...")
#stopWordsBroadcast.value
#print("Stop words End...")

end_time = time.time()

print("Execution Time taken(in seconds) to create RDD's:", end_time-start_time)




Execution Time taken(in seconds) to create RDD's: 0.40788912773132324


#Exercise 1. 3 (Joining collections)

In [5]:
#to record execution time
start_time = time.time()

#load stopwords_en file
stopWords = sc.textFile("stopwords_en.txt")
stopWordsBroadcast = sc.broadcast(stopWords.collect())

# Finding top 10 most frequent words
def FrequentWords(x):
    rList = Counter(x)
    FrequentWordsCount = CounterList.most_common(10)
    FrequentWords = [word for word, word_count in FrequentWordsCount]
    return FrequentWords

#to Remove the stopWords
def removeStopWords(List):
    requiredList = List.copy()
    for w in List:
        if ((w in stopWordsBroadcast.value) or a == "" or a == " "):
            requiredList.remove(a)
    return requiredList


# join the RDDs
userLibrariesJoinPapersRdd = userRatingsRDD.flatMapValues(lambda x: x).map(lambda x: (x[1],x[0])).join(paperTermsRDD)
#restructure the RDD
userLibrariesJoinPapersRdd = userLibrariesJoinPapersRdd.map(lambda x: (x[1][0],x[1][1]))
#transformation of RDD to perform computations 
userLibrariesJoinPapersRdd = userLibrariesJoinPapersRdd.flatMapValues(lambda x:x).groupByKey().mapValues(list)
#to remove the stop words from the RDD
userLibrariesJoinPapersRddWithoutStopWords = userLibrariesJoinPapersRdd.mapValues(removeStopWords)

# now run the def frequentWords after the stop words are removed
frequentWordList = userLibrariesJoinPapersRddWithoutStopWords.mapValues(FrequentWords)

#wrtiting the above data into file
def CreateCsv(data):
    Data = data[0] + "," + (','.join(str(d) for d in data[1]))
    return Data

frequentWordListFile = frequentWordList.map(CreateCsv)
frequentWordListFile.saveAsTextFile("Results")

end_time = time.time()
print("Execution Time taken for Joining collections and writing it to a file:", end_time-start_time, "seconds")

Py4JJavaError: An error occurred while calling o111.saveAsTextFile.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.internal.io.SparkHadoopWriter$.write(SparkHadoopWriter.scala:106)
	at org.apache.spark.rdd.PairRDDFunctions.$anonfun$saveAsHadoopDataset$1(PairRDDFunctions.scala:1091)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:112)
	at org.apache.spark.rdd.RDD.withScope(RDD.scala:405)
	at org.apache.spark.rdd.PairRDDFunctions.saveAsHadoopDataset(PairRDDFunctions.scala:1089)
	at org.apache.spark.rdd.PairRDDFunctions.$anonfun$saveAsHadoopFile$4(PairRDDFunctions.scala:1062)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.rdd.RDDOperationScope$.withScop

#Exercise 1. 4 (Basic Analysis for Recommender Systems)

In [6]:



#to record execution time
start_time = time.time()

#a) Number of (distinct) user, number of (distinct) items, and number of ratings

#first apply flatMapVales() function for each kay-pair value in userRatingsRDD
userRatingsRDDFMV =userRatingsRDD.flatMapValues(lambda x:x)

numOfUsers = userRatingsRDDFMV.keys().distinct().count()
print("Number of distinct users: ", numOfUsers)

numOfItems = userRatingsRDDFMV.values().distinct().count()
print("Number of distinct items: ", numOfItems)

numOfRatings = userRatingsRDDFMV.count()
print("Number of ratings: ", numOfRatings)

#b) Min number of ratings a user has given
minNumberOfRatings = userRatingsRDD.map(lambda x: (x[0],len(x[1]))).map(lambda x: x[1]).min()
print("Minimum number of ratings a user has given: ", minNumberOfRatings)

#c)  Max number of ratings a user has given
maxNumberOfRatings = userRatingsRDD.map(lambda x: (x[0],len(x[1]))).map(lambda x: x[1]).max()
print("Maximum number of ratings a user has given: ", maxNumberOfRatings)

#d)  Average number of ratings of users
avgNumberOfRatings = numOfRatings/numOfUsers
print("Average number of ratings a user has given: ", avgNumberOfRatings)

#e) Standard deviation for ratings of users
stdevRatingsOfUsers = (userRatingsRDD.map(lambda x: (x[0],len(x[1]))).map(lambda x: x[1])).stdev()
print("Standard deviation of ratings for users",stdevRatingsOfUsers)
 
#f)Min number of ratings an item has received
minRatingsOfitem = userRatingsRDDFMV.map(lambda x: (x[1],1)).reduceByKey(lambda x,y: x+y).map(lambda x: x[1]).min()
print("Minimum number of ratings an item has received: ", minRatingsOfitem)

#g) Max number of ratings an item has received
maxRatingsOfitem = userRatingsRDDFMV.map(lambda x: (x[1],1)).reduceByKey(lambda x,y: x+y).map(lambda x: x[1]).max()
print("Maximum number of ratings an item has received: ", maxRatingsOfitem)

#h) Average number of ratings of items
avgRatingsOfitem= numOfRatings/numOfItems
print("Average number of ratings per item: ", avgRatingsOfitem)

#i) Standard deviation for ratings of items
stdevRatingsOfItems = userRatingsRDDFMV.map(lambda x: (x[1],1)).reduceByKey(lambda x,y: x+y).map(lambda x: x[1]).stdev()
print("Standard deviation of ratings per item: ", stdevRatingsOfItems)

end_time = time.time()
print("Execution Time taken for Basic Analysis for Recommender Systems:", end_time-start_time,"seconds")

Number of distinct users:  28416
Number of distinct items:  172079
Number of ratings:  828481
Minimum number of ratings a user has given:  1
Maximum number of ratings a user has given:  1922
Average number of ratings a user has given:  29.155440596846848
Standard deviation of ratings for users 81.1751761366871
Minimum number of ratings an item has received:  3
Maximum number of ratings an item has received:  924
Average number of ratings per item:  4.81453867119172
Standard deviation of ratings per item:  5.477802292314525
Execution Time taken for Basic Analysis for Recommender Systems: 5.430299282073975 seconds


#Exercise 1. 5 (Loading the dataset into Data Frames)

In [7]:




# To define the schema of the CSV data
papersSchema = StructType([
    StructField("paper_id", IntegerType(), True),
    StructField("type", StringType(), True),
    StructField("journal", StringType(), True),
    StructField("bookـtitle", StringType(), True),
    StructField("series", IntegerType(), True),
    StructField("publisher", StringType(), True),
    StructField("pages", IntegerType(), True),
    StructField("volume", IntegerType(), True),
    StructField("number", IntegerType(), True),
    StructField("year", IntegerType(), True),
    StructField("month", StringType(), True),
    StructField("postedat", FloatType(), True),
    StructField("address", StringType(), True),
    StructField("title", StringType(), True),
    StructField("abstract", StringType(), True)
    ])
df_papers = sparkSession.read.csv("papers.csv", sep = ",", header = True, schema = papersSchema)
df_papers = df_papers.na.drop(subset=["abstract"]) # drops the null values

# To define the schema of the text data
userLibrariesSchema = StructType([
    StructField("user_hash_id",StringType(),True),
    StructField("user_library",StringType(),True)
])
df_userLibraries = sparkSession.read.csv("users_libraries.txt", sep = ";", header = True, schema = userLibrariesSchema)
# Select the columns using selectExpr() function
df_userLibraries = df_userLibraries.selectExpr("user_hash_id","split(user_library,',') AS user_library")





df_userLibraries.head()

df_papers.head()


end_time = time.time()

print("Execution Time taken(in seconds) to create Dataframes:", end_time-start_time)

Execution Time taken(in seconds) to create Dataframes: 8.308101654052734


#Ex1.6 - Using Dataframes for Ex1.3

In [8]:

start_time = time.time()

#Explode UserLibrary
userLibraryExplode = df_userLibraries.select(df_userLibraries.user_hash_id,explode(df_userLibraries.user_library).alias("paper_id"))

#Join UserLibrary and PaperCsv
df_userLibraryJoinPaperCsv = df_papers.join(userLibraryExplode,df_papers.paper_id == userLibraryExplode.paper_id, how="inner").select(userLibraryExplode.user_hash_id,userLibraryExplode.paper_id,df_papers.abstract)


# Define the useless words list
useless_words = ['',' ','"']

# Filter out rows with empty or useless abstracts
df_userLibraryJoinPaperCsv = df_userLibraryJoinPaperCsv.filter(
    (~col("abstract").isin(useless_words)) &
    (col("abstract").isNotNull())
)

# Define a UDF to remove stop words from the abstract
def remove_stop_words(abstract):
    words = abstract.split(" ")
    cleaned_words = [word for word in words if word.lower() not in stopWordsBroadcast.value]
    return " ".join(cleaned_words)

# Register the UDF with Spark
remove_stop_words_udf = udf(remove_stop_words, StringType())

# Apply the UDF to the "abstract" column
df_userLibraryJoinPaperCsvWithoutStopWords = df_userLibraryJoinPaperCsv.withColumn("abstract", remove_stop_words_udf(col("abstract")))

# Now df_userLibraryJoinPaperCsvWithoutStopWords contains the DataFrame with stop words removed from the "abstract" column.


#Finding top 10 most frequent words
df_userLibraryJoinPaperCsvWithoutStopWordsCount = df_userLibraryJoinPaperCsvWithoutStopWords.groupBy("user_hash_id","abstract").count().withColumnRenamed("count", "word_count")
userWords_window = Window.partitionBy(df_userLibraryJoinPaperCsvWithoutStopWordsCount.user_hash_id).orderBy(col("word_count").desc())
df_userLibraryJoinPaperCsvWithoutStopWordsRank = df_userLibraryJoinPaperCsvWithoutStopWordsCount.withColumn("word_rank",rank().over(userWords_window))
df_topFrequentWordsPerUser = df_userLibraryJoinPaperCsvWithoutStopWordsRank.filter(df_userLibraryJoinPaperCsvWithoutStopWordsRank["word_rank"]<11)
df_groupedTop10FrequentWordsPerUser = df_topFrequentWordsPerUser.groupBy("user_hash_id").agg(collect_list("abstract")).withColumnRenamed("collect_list(abstract)", "abstract_word_list")

#writing top 10 most frequent words of each user to file
df_groupedTop10FrequentWordsPerUser.write.save("Outputs/Top10WordsForEachUser_DF")

#print("Top 10 most frequent words of each user_hash_id Start...")
#df_groupedTop10FrequentWordsPerUser.take(10)
#print("Top 10 most frequent words of each user_hash_id End...")

end_time = time.time()

print("Execution Time taken(in seconds) for Joining collections and writing to file", end_time-start_time)





Execution Time taken(in seconds) for Joining collections and writing to file 186.3451726436615


#Ex1.6 - Using Dataframes for Ex1.4

In [9]:

from pyspark.sql import SparkSession
from pyspark.sql.functions import count, countDistinct, avg, stddev





# a
noOfDistinctUsers_df = userLibraryExplode.select(countDistinct("user_hash_id")).collect()[0][0]
noOfDistinctItems_df = userLibraryExplode.select(countDistinct("paper_id")).collect()[0][0]
noOfRatings_df = userLibraryExplode.count()

# b, c, d, e
ratingsList_df = userLibraryExplode.groupBy("user_hash_id").count().withColumnRenamed("count", "no_of_items")
ratingsList_df = ratingsList_df.describe("no_of_items")

summary_df = ratingsList_df.select("summary", "no_of_items").rdd.collectAsMap()

minNoOfRatingUserHasGiven = summary_df["min"]
maxNoOfRatingUserHasGiven = summary_df["max"]
avgNumberOfRatingUserGave = noOfRatings_df / noOfDistinctUsers_df
standardDeviationOfRating = summary_df["stddev"]

# f, g, h, i
ratingsListByPaperId_df = userLibraryExplode.groupBy("paper_id").count().withColumnRenamed("count", "no_of_ratings")
ratingsListByPaperId_df = ratingsListByPaperId_df.describe("no_of_ratings")

summary_df = ratingsListByPaperId_df.select("summary", "no_of_ratings").rdd.collectAsMap()

minNoOfRatingItemHasReceived_df = summary_df["min"]
maxNoOfRatingItemHasReceived_df = summary_df["max"]
avgNumberOfRatingOfItems_df = noOfRatings_df / noOfDistinctItems_df
standardDeviationOfRItem_df = summary_df["stddev"]

# Stop the Spark session
#spark.stop()


print("Number of (distinct) user:" ,noOfDistinctUsers_df)
print("Number of (distinct) items:" ,noOfDistinctItems_df)
print("Number of ratings:" ,noOfRatings_df)
print("Min number of ratings a user has given:",minNoOfRatingUserHasGiven)
print("Max number of ratings a user has given:",maxNoOfRatingUserHasGiven)
print("Average number of ratings of users:",avgNumberOfRatingUserGave)
print("Standard deviation for ratings of users:",standardDeviationOfRating)
print("Min number of ratings an item has received:",minNoOfRatingItemHasReceived_df)
print("Max number of ratings an item has received:",maxNoOfRatingItemHasReceived_df)
print("Average number of ratings of items:",avgNumberOfRatingOfItems_df)
print("Standard deviation for ratings of items:",standardDeviationOfRItem_df)

end_time = time.time()

print("Execution Time taken(in seconds) for Basic Analysis for Recommender Systems:", end_time-start_time)

Number of (distinct) user: 28415
Number of (distinct) items: 172079
Number of ratings: 828461
Min number of ratings a user has given: 1
Max number of ratings a user has given: 1922
Average number of ratings of users: 29.15576280133732
Standard deviation for ratings of users: 81.17801478819581
Min number of ratings an item has received: 2
Max number of ratings an item has received: 924
Average number of ratings of items: 4.814422445504681
Standard deviation for ratings of items: 5.47783230760693
Execution Time taken(in seconds) for Basic Analysis for Recommender Systems: 195.0146381855011
